In [1]:
import copy
import os.path as osp

import click
import cv2
import matplotlib.cm as cm
import numpy as np
import torch
import torch.hub
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models, transforms

# from grad_cam import (
#     BackPropagation,
#     Deconvnet,
#     GradCAM,
#     GuidedBackPropagation,
#     occlusion_sensitivity,
# )

In [2]:
def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device

In [3]:
def get_classtable():
    classes = []
    with open("samples/synset_words.txt") as lines:
        for line in lines:
            line = line.strip().split(" ", 1)[1]
            line = line.split(", ", 1)[0].replace(" ", "_")
            classes.append(line)
    return classes

In [7]:
def preprocess(image_path):
    raw_image = cv2.imread(image_path)
    raw_image = cv2.resize(raw_image, (224,) * 2)
    image = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    return image, raw_image

In [8]:
def save_gradient(filename, gradient):
    gradient = gradient.cpu().numpy().transpose(1, 2, 0)
    gradient -= gradient.min()
    gradient /= gradient.max()
    gradient *= 255.0
    cv2.imwrite(filename, np.uint8(gradient))

In [9]:
def save_gradcam(filename, gcam, raw_image, paper_cmap=False):
    gcam = gcam.cpu().numpy()
    cmap = cm.jet_r(gcam)[..., :3] * 255.0
    if paper_cmap:
        alpha = gcam[..., None]
        gcam = alpha * cmap + (1 - alpha) * raw_image
    else:
        gcam = (cmap.astype(np.float) + raw_image.astype(np.float)) / 2
    cv2.imwrite(filename, np.uint8(gcam))

In [10]:
def save_sensitivity(filename, maps):
    maps = maps.cpu().numpy()
    scale = max(maps[maps > 0].max(), -maps[maps <= 0].min())
    maps = maps / scale * 0.5
    maps += 0.5
    maps = cm.bwr_r(maps)[..., :3]
    maps = np.uint8(maps * 255.0)
    maps = cv2.resize(maps, (224, 224), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename, maps)

In [12]:
device = get_device(True)

Device: GeForce GTX 1070


In [13]:
classes = get_classtable()

FileNotFoundError: [Errno 2] No such file or directory: 'samples/synset_words.txt'

In [2]:
model = torch.load("resnet50_12epoch_model")

In [3]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
from PIL import Image 
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
trainLabels = pd.read_csv("./trainLabels.csv")

In [9]:
train_path = './train/train_224/*'
images_list = glob.glob(train_path)
images_list = sorted(images_list)

In [10]:
def get_input(idx):
        imgtemp = Image.open(images_train_list[idx])
#         if self.transform!=None :
#             imgtemp = self.transform(imgtemp)
        img = transforms.ToTensor()(imgtemp)
        fileName = images_train_list[idx].split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[trainLabels.loc[trainLabels.image==fileName ,'level'].values[0]] = 1
        return img ,imglabel

In [11]:
def get_pred_for_one_image(idx,model):
        test , test_label = get_input(idx)
        test = test.view(-1,3,224,224)
        test_label = test_label.view(-1,5)
        test = test.to(device)
        test_label = test_label.to(device)
        print("test_label",test_label)
        pred = model(test)
        print("pred",pred)

In [12]:
def get_labels(filenames,trainLabels):
    labels = []
    for file in filenames:
        ft = file.split('/')[-1].split('.')[0]
        imglabel = torch.zeros(5,dtype=torch.long)
        imglabel[trainLabels.loc[trainLabels.image==ft ,'level'].values[0]] = 1
        labels.append(imglabel)
    return labels

In [13]:
f = open("train_balanced_name","r")

In [14]:
lines = f.readlines()

In [15]:
total_balanced_images = []
for line in lines:
    temp = line.strip()
    temp = './train/train_224/' + temp
    total_balanced_images.append(temp)

In [16]:
class_indices = {0: [] , 1: [] , 2: [] , 3 : [], 4:[]}
for idx in range(len(total_balanced_images)):
    fileName = total_balanced_images[idx].split('/')[-1].split('.')[0]
    label = torch.zeros(5,dtype=torch.long)
    label[trainLabels.loc[trainLabels.image==fileName ,'level'].values[0]] = 1
#     print(label)
    _, img_tru_class = torch.max(label,0)
#     print(img_tru_class)
    if idx%1000==0:
        print("idx ",idx)
    class_indices[img_tru_class.item()].append(idx)

idx  0
idx  1000
idx  2000
idx  3000
idx  4000
idx  5000
idx  6000
idx  7000
idx  8000
idx  9000
idx  10000
idx  11000
idx  12000
idx  13000


In [17]:
class_indices[3]

[7670,
 7671,
 7672,
 7673,
 7674,
 7675,
 7676,
 7677,
 7678,
 7679,
 7680,
 7681,
 7682,
 7683,
 7684,
 7685,
 7686,
 7687,
 7688,
 7689,
 7690,
 7691,
 7692,
 7693,
 7694,
 7695,
 7696,
 7697,
 7698,
 7699,
 7700,
 7701,
 7702,
 7703,
 7704,
 7705,
 7706,
 7707,
 7708,
 7709,
 7710,
 7711,
 7712,
 7713,
 7714,
 7715,
 7716,
 7717,
 7718,
 7719,
 7720,
 7721,
 7722,
 7723,
 7724,
 7725,
 7726,
 7727,
 7728,
 7729,
 7730,
 7731,
 7732,
 7733,
 7734,
 7735,
 7736,
 7737,
 7738,
 7739,
 7740,
 7741,
 7742,
 7743,
 7744,
 7745,
 7746,
 7747,
 7748,
 7749,
 7750,
 7751,
 7752,
 7753,
 7754,
 7755,
 7756,
 7757,
 7758,
 7759,
 7760,
 7761,
 7762,
 7763,
 7764,
 7765,
 7766,
 7767,
 7768,
 7769,
 7770,
 7771,
 7772,
 7773,
 7774,
 7775,
 7776,
 7777,
 7778,
 7779,
 7780,
 7781,
 7782,
 7783,
 7784,
 7785,
 7786,
 7787,
 7788,
 7789,
 7790,
 7791,
 7792,
 7793,
 7794,
 7795,
 7796,
 7797,
 7798,
 7799,
 7800,
 7801,
 7802,
 7803,
 7804,
 7805,
 7806,
 7807,
 7808,
 7809,
 7810,
 7811,
 7812,

In [59]:
path = total_balanced_images[class_indices[1][20]]
img = Image.open(path)
img.show()

In [60]:
path

'./train/train_224/10320_right.tiff'

'./train/train_224/10510_left.tiff' -> 3 <br>
'./train/train_224/20500_right.tiff' -> 2 <br>
'./train/train_224/31712_right.tiff' -> 3 <br>
'./train/train_224/31758_left.tiff' -> 3 <br>
'./train/train_224/42265_left.tiff' -> 2 <br>
'./train/train_224/18819_left.tiff' -> 4 <br>
'./train/train_224/36636_right.tiff' -> 2 <br>
'./train/train_224/14016_right.tiff' -> 0 <br>
'./train/train_224/10320_right.tiff' -> 1

In [61]:
img = Image.open('./train/train_224/42265_left.tiff')
img.show()